In [1]:
!pip install lughaatNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from collections import Counter
import re
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [21]:
import pandas as pd
from LughaatNLP import LughaatNLP

urdu_text_processing = LughaatNLP()

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Urdu Poetry/Ghazal_ur.csv')

# Function to clean and normalize text
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = urdu_text_processing.remove_special_characters(text)  # Remove punctuation
    text = urdu_text_processing.remove_diacritics(text)  # Remove diacritics
    text = urdu_text_processing.remove_whitespace(text)  # Remove extra spaces
    text = urdu_text_processing.normalize(text)  # Apply full normalization
    return text

# Apply cleaning function to both columns
df['misra1_clean'] = df['misra1'].apply(clean_text)
df['misra2_clean'] = df['misra2'].apply(clean_text)

# Display cleaned data
print(df[['misra1_clean', 'misra2_clean']].head())

                                misra1_clean  \
0      ہم نے اک عمر بسر کی ہے غم یار کے ساتھ   
1  اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں   
2   اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ   
3       ایک تو خواب لیے پھرتے ہو گلیوں گلیوں   
4      شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا   

                                 misra2_clean  
0       میرؔ دو دن نہ جئے ہجر کے ازار کے ساتھ  
1          طاق پر عزت سادات بھی دستار کے ساتھ  
2   چاپ سنتے ہیں تو لگ جاتے ہیں دیوار کے ساتھ  
3      اس پہ تکرار بھی کرتے ہو خریدار کے ساتھ  
4  ورنہ ہم رند تو بھڑ جاتے ہیں دو چار کے ساتھ  


In [ ]:

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Urdu Poetry/Ghazal_ur.csv')

def clean_urdu_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'اآ', 'ا', text)
    text = re.sub(r'ء', 'ٔ', text)
    text = re.sub(r'ی', 'ي', text)
    text = re.sub(r'ک', 'ک', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['misra1_clean'] = df['misra1'].apply(clean_urdu_text)
df['misra2_clean'] = df['misra2'].apply(clean_urdu_text)


In [22]:

# Tokenization
def tokenize(text):
    return text.split()

df['misra1_tokens'] = df['misra1_clean'].apply(tokenize)
df['misra2_tokens'] = df['misra2_clean'].apply(tokenize)

# Building Vocabulary
all_tokens = [token for tokens in df['misra1_tokens'].tolist() + df['misra2_tokens'].tolist() for token in tokens]
vocab = Counter(all_tokens)
vocab = {word: i+1 for i, (word, _) in enumerate(vocab.most_common())}  # 1-based indexing
vocab['<PAD>'] = 0  # Padding token

def encode(tokens):
    return [vocab[token] for token in tokens if token in vocab]

df['misra1_encoded'] = df['misra1_tokens'].apply(encode)
df['misra2_encoded'] = df['misra2_tokens'].apply(encode)

# Padding sequences
max_len = max(df['misra1_encoded'].apply(len).max(), df['misra2_encoded'].apply(len).max())

def pad_sequence_custom(seq):
    return seq + [0] * (max_len - len(seq))

df['misra1_padded'] = df['misra1_encoded'].apply(pad_sequence_custom)
df['misra2_padded'] = df['misra2_encoded'].apply(pad_sequence_custom)


In [23]:

# Creating Dataset
class GhazalDataset(Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df['misra1_padded'].tolist(), dtype=torch.long)
        self.y = torch.tensor(df['misra2_padded'].tolist(), dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_dataset = GhazalDataset(train_df)
val_dataset = GhazalDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [24]:
# Defining LSTM Model with Repetition Control
class LSTMGhazalGenerator(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=2, repetition_penalty=1.2):
        super(LSTMGhazalGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.repetition_penalty = repetition_penalty  # Added repetition penalty

    def forward(self, x, past_tokens=None):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.fc(lstm_out)

        # Apply repetition penalty
        if past_tokens is not None:
            for idx in range(logits.shape[1]):  # Loop over sequence length
                for token in past_tokens:
                    logits[:, idx, token] /= self.repetition_penalty  # Penalize repeated words

        return logits


In [27]:

# Model Training
vocab_size = len(vocab)
model = LSTMGhazalGenerator(vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding tokens
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, epochs=140):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output.view(-1, vocab_size), y_batch.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
train_model(model, train_loader, val_loader)


Epoch 1: 100%|██████████| 381/381 [00:03<00:00, 121.44it/s]


Epoch 1, Loss: 7.0281626921626215


Epoch 2: 100%|██████████| 381/381 [00:03<00:00, 126.56it/s]


Epoch 2, Loss: 6.651762430436342


Epoch 3: 100%|██████████| 381/381 [00:03<00:00, 120.80it/s]


...

Epoch 138, Loss: 0.44722911965815726


Epoch 139: 100%|██████████| 381/381 [00:03<00:00, 122.47it/s]


Epoch 139, Loss: 0.4485325238757872


Epoch 140: 100%|██████████| 381/381 [00:03<00:00, 119.65it/s]

Epoch 140, Loss: 0.44968357025169015


In [33]:

# Inference
import random

def generate_ghazal(input_misra, model, vocab, max_lines=1):
    model.eval()
    tokens = tokenize(clean_text(input_misra))
    encoded = torch.tensor(pad_sequence_custom(encode(tokens)), dtype=torch.long).unsqueeze(0).to(device)
    generated_ghazal = [input_misra]

    for _ in range(max_lines):
        with torch.no_grad():
            output = model(encoded)
            predicted_ids = torch.argmax(output, dim=2).squeeze(0).tolist()
        predicted_tokens = [list(vocab.keys())[list(vocab.values()).index(i)] for i in predicted_ids if i != 0]
        generated_misra = " ".join(predicted_tokens)
        generated_ghazal.append(generated_misra)
        encoded = torch.tensor(pad_sequence_custom(encode(predicted_tokens)), dtype=torch.long).unsqueeze(0).to(device)
    return "\n".join(generated_ghazal)

# Example Usage
input_misra = input("برائے مہربانی ایک مصرع درج کریں: ")
print(generate_ghazal(input_misra, model, vocab))


برائے مہربانی ایک مصرع درج کریں: چاندنی راتوں میں تنہا بیٹھا ہوں
چاندنی راتوں میں تنہا بیٹھا ہوں
اج بھی میرے صبح و میرا گیا ہے گا ایا ایا ایا ایا پایا نکلے نکلے گے پایا


In [31]:
# Save the model
model_path = "/content/drive/MyDrive/Urdu Poetry/ghazal_generator.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")

Model saved at /content/drive/MyDrive/Urdu Poetry/ghazal_generator.pth
